# Formatting Data (dates)

It is very common to find dates (some combination of year, month, day of week and time) in data that is collected in real time (and other that organize event information.

Let's see a data frame that comes with dates from an API.

In [1]:
import pandas as pd
from sodapy import Socrata

client = Socrata("data.seattle.gov", None)

results = client.get("kzjm-xkqj", limit=2000)

# Convert to pandas DataFrame
calls911 = pd.DataFrame.from_records(results)

Let's check some information:

In [2]:
calls911.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   address                      2000 non-null   object
 1   type                         2000 non-null   object
 2   datetime                     2000 non-null   object
 3   latitude                     1996 non-null   object
 4   longitude                    1996 non-null   object
 5   report_location              1997 non-null   object
 6   incident_number              2000 non-null   object
 7   :@computed_region_ru88_fbhk  1992 non-null   object
 8   :@computed_region_kuhn_3gp2  1992 non-null   object
 9   :@computed_region_q256_3sug  1997 non-null   object
 10  :@computed_region_2day_rhn5  176 non-null    object
 11  :@computed_region_cyqu_gs94  165 non-null    object
dtypes: object(12)
memory usage: 187.6+ KB


Let's get rid of some columns:

In [3]:
calls911=calls911.iloc[:,:7]

Let's check the column _datetime_:

In [4]:
calls911.datetime.head()

0    2022-08-18T10:18:00.000
1    2022-08-18T10:16:00.000
2    2022-08-18T10:16:00.000
3    2022-08-18T10:15:00.000
4    2022-08-18T10:08:00.000
Name: datetime, dtype: object

In [5]:
# then
type(calls911.datetime[0])


str

The date and time information is not useful at this time, that is, the information it offers is of limited use, as it is just a string. 

Let's make it useful:

In [6]:
calls911.datetime=pd.to_datetime(calls911.datetime,format='%Y-%m-%d')
calls911.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   address          2000 non-null   object        
 1   type             2000 non-null   object        
 2   datetime         2000 non-null   datetime64[ns]
 3   latitude         1996 non-null   object        
 4   longitude        1996 non-null   object        
 5   report_location  1997 non-null   object        
 6   incident_number  2000 non-null   object        
dtypes: datetime64[ns](1), object(6)
memory usage: 109.5+ KB


Once you have this data type, you can retrieve important information:

In [7]:
calls911['date']=calls911.datetime.dt.date
calls911['year']=calls911.datetime.dt.year
calls911['month']=calls911.datetime.dt.month_name()
calls911['weekday']=calls911.datetime.dt.day_name()
calls911['hour']=calls911.datetime.dt.hour

In [8]:
calls911.head()

,address,type,datetime,latitude,longitude,report_location,incident_number,date,year,month,weekday,hour
0,450 Ne 100th St,Aid Response,2022-08-18 10:18:00,47.701329,-122.323684,"{'type': 'Point', 'coordinates': [-122.323684,...",F220099506,2022-08-18,2022,August,Thursday,10
1,1020 Seneca St,Aid Response,2022-08-18 10:16:00,47.610396,-122.327328,"{'type': 'Point', 'coordinates': [-122.327328,...",F220099504,2022-08-18,2022,August,Thursday,10
2,S Alaska St / M L King Jr Way S,BC Aid Response,2022-08-18 10:16:00,47.560808,-122.29324,"{'type': 'Point', 'coordinates': [-122.29324, ...",F220099505,2022-08-18,2022,August,Thursday,10
3,4th Ave N / Mercer St,Triaged Incident,2022-08-18 10:15:00,47.624564,-122.348877,"{'type': 'Point', 'coordinates': [-122.348877,...",F220099503,2022-08-18,2022,August,Thursday,10
4,815 S Dearborn St,Investigate Out Of Service,2022-08-18 10:08:00,47.595831,-122.322292,"{'type': 'Point', 'coordinates': [-122.322292,...",F220099502,2022-08-18,2022,August,Thursday,10


Let's create a new column with what we have. In this case, a boolean where you tell if it is night time (after 8 pm before 6 am):

In [9]:
calls911['nightTime']=((calls911['hour']<=6) | (calls911['hour']>=20))

Let's save what we have:

In [10]:
import os

where=os.path.join('DataFiles','calls911.pkl')
calls911.to_pickle(where)